In [7]:
# Read csv
from tqdm import tqdm
from collections import defaultdict
import csv

champions = {}
#with open('/Users/tonykim/Downloads/champions-CCs,lanes.csv') as f:
with open('new_champion_V1.csv') as f:
    reader = csv.DictReader(f)
    for champion in tqdm(reader):
        champions[int(champion['key'])] = champion

with open('/Users/tonykim/Downloads/merged_match_dataset_V0.csv') as f:
    reader = csv.DictReader(f)
    rows = [row for row in tqdm(reader)]

146it [00:00, 37084.02it/s]
350880it [00:16, 20697.38it/s]


In [ ]:
champion_picks_rows = defaultdict(list) # key -> list
champion_bans_rows = defaultdict(list)

for row in tqdm(rows):
    champion_picks_rows[int(row['championId_Pick'])].append(row)
    champion_bans_rows[int(row['championId_Ban'])].append(row)

In [ ]:
def calculate_kda(k, d, a):
    if d == 0:
        return k + a
    return (k + a) / d

In [ ]:
for c_id, champion in champions.items():
    # Select rows
    pick_rows = champion_picks_rows[c_id]
    ban_rows = champion_bans_rows[c_id]
    
    # Calculate winrate
    win_rate = sum([1 for row in pick_rows if row['win_x'] == 'Win']) / len(pick_rows)
    print(f'Win rate for champion {champion["name"]} is {win_rate*100:.4f}')
    
    # Calculate pickrate
    # picks = # of filtered_rows
    # total_games = rows/10
    # pick_rate = picks / total_games
    total_games = (len(rows) // 10)
    pick_rate = len(pick_rows) / total_games
    ban_rate = len(ban_rows) / total_games
    pick_ban_rate = pick_rate + ban_rate
    print(f'Pick rate for champion {champion["name"]} is {pick_rate*100:.04f}')
    print(f'Ban rate for champion {champion["name"]} is {ban_rate*100:.04f}')
    print(f'PickBan rate is {pick_ban_rate*100:.04f}')
    
    # Calculate KDA
    kda = calculate_kda(sum(int(row['kills']) for row in pick_rows), sum(int(row['deaths']) for row in pick_rows), sum(int(row['deaths']) for row in pick_rows))
    print(f'KDA for champion {champion["name"]} is {kda:.04f}')
    
    # Store to champions dataset
    champion['win_rate'] = win_rate
    champion['pick_rate'] = pick_rate
    champion['ban_rate'] = ban_rate
    champion['pick_ban_rate'] = pick_ban_rate
    champion['average_KDA'] = kda

In [ ]:
# Sanity check
sum(champion['pick_rate'] for champion in champions.values()) # Should be 10.0

In [ ]:
# Save results
with open('new_champions_rates_V1.csv', 'w') as output:
    writer = csv.DictWriter(output, fieldnames=champions[266].keys())
    writer.writeheader()
    for champion in champions.values():
        writer.writerow(champion)

# Matt's code for calculating runes/items

In [1]:
import pandas as pd
import csv
import statistics

In [2]:
match_data = pd.read_csv('merged_match_dataset_V1.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33,34,104,138) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def get_top_item(item_counts):
    index_ = str(list(item_counts.values()).index(max(list(item_counts.values()))))
    item_counts[list(item_counts.keys())[int(index_)]] = -1
    return str(list(item_counts.keys())[int(index_)])

def top_3(variable_list, variable):
    var_set = set(variable_list)
    var_counts = dict()
    for entry in var_set:
        var_counts[entry] = variable_list.count(entry)
        
    top_3_entries = []
    for i in range(4):
        top_3_entries.append(get_top_item(var_counts))
        
    # For when the player did not select an item
    if '0' in top_3_entries:
        #items = sorted(items)
        top_3_entries.pop(top_3_entries.index("0"))
    if variable == 'Allies':
        top_3_entries.pop(0)
    return top_3_entries #top_3_df

def get_champ_stats(champion, data):

    champion_difficulty = data[(data['key'] == champion.iloc[0]['championId_Pick'])]['info.difficulty']
    champion_difficulty = int(pd.DataFrame(champion_difficulty).iloc[0][0])
    
    # When we are only concerned with games that the champion won
    champion_wins = champion[(champion['win_y'] == True)]
#     print("Champion Difficulty")
#     print(champion_difficulty)
    
    # Top Items
    item0 = list(champion_wins['item0'])
    item1 = list(champion_wins['item1'])
    item2 = list(champion_wins['item2'])
    item3 = list(champion_wins['item3'])
    item4 = list(champion_wins['item4'])
    item5 = list(champion_wins['item5'])
    item6 = list(champion_wins['item6'])
    
    # Top Perks
    perk0 = list(champion_wins['perk0'])
    perk1 = list(champion_wins['perk1'])
    perk2 = list(champion_wins['perk2'])
    perk3 = list(champion_wins['perk3'])
    perk4 = list(champion_wins['perk4'])
    perk5 = list(champion_wins['perk5'])
    
    # Top Spells
    spell_1 = list(champion_wins['spell1Id'])
    spell_2 = list(champion_wins['spell2Id'])
    
    all_perks = perk0 + perk1 + perk2 + perk3 + perk4 + perk5
    all_items = item0 + item1 + item2 + item3 +item4 + item5 + item5
    all_spells = spell_1 + spell_2
    

    
    all_winning_games =  get_WL_games(champion.iloc[0]['championId_Pick'],match_data, True) # This is all the winning games 
    team_mates = all_winning_games[(all_winning_games['win_y'] == True)] # DataFrame where we get all the champions that won with the selected champion
    all_champs = list(team_mates['championId_Pick']) # This is all of the champions

    

    counters = all_winning_games[(all_winning_games['win_y'] == False)]
    all_counters = list(counters['championId_Pick'])
    
    runes = top_3(all_perks, "Runes")
    items = top_3(all_items, "Items")
    spells= top_3(all_spells, "Spells")
    allies = top_3(all_champs, "Allies")
    top_counters = top_3(all_counters, "Counters")
    top_stats = pd.DataFrame()
    for i in range(3):
        current_champ_stats = pd.DataFrame([[runes[i], items[i], spells[i], allies[i], top_counters[i]]], columns=["Runes"+str(i), "Items"+str(i), "Spells"+str(i), "Allies"+str(i), "Counters"+str(i)])
        top_stats = pd.concat([top_stats, current_champ_stats], axis = 1)
    
    
    return top_stats
    
def get_champion_games(champ_id, data):
    games = data[(data['championId_Pick'] == champ_id)]['gameId']
    games = list(games)
    matches = pd.DataFrame()
    for game in games:
        match_data = data[(data['gameId'] == game)]
        matches = pd.concat([match_data, matches], axis=0)
    return matches

def get_WL_games(champ_id, data, status):
    games = data[(data['championId_Pick'] == champ_id) & (data['win_y'] == status)]['gameId']
    games = list(games)
    matches = pd.DataFrame()
    for game in games:
        match_data = data[(data['gameId'] == game)]
        matches = pd.concat([match_data, matches], axis=0)
    return matches

In [4]:
champion_data = pd.read_csv("/Users/tonykim/Downloads/champions-CCs,lanes.csv")
champion_ids = sorted(list(champion_data['key']))

champ_df = match_data[(match_data['championId_Pick']==2)]

#get_champ_stats(champ_df, champion_data)

In [8]:
stats_df = pd.DataFrame()
for champion in tqdm(champion_ids):
    try:
        entire_champ_df = pd.DataFrame()
        champ_df = match_data[(match_data['championId_Pick']==champion)]
        champ_stats = get_champ_stats(champ_df, champion_data)
        entire_champ_df = pd.concat([champ_stats, entire_champ_df])
        champ_id = pd.DataFrame([[champion]], columns=['id'])
        entire_champ_df = pd.concat([champ_id, entire_champ_df], axis= 1)
        stats_df = pd.concat([entire_champ_df, stats_df], axis=0)
    except KeyError:
        print("Dont have champion: " + str(champion) )
stats_df.to_csv(path_or_buf='champion_stats.csv')
stats_df

100%|██████████| 146/146 [1:22:13<00:00, 22.89s/it] 


,id,Runes0,Items0,Spells0,Allies0,Counters0,Runes1,Items1,Spells1,Allies1,Counters1,Runes2,Items2,Spells2,Allies2,Counters2
0,555,8439,3117,4,145,235,8473,3147,14,245,51,8446,3142,3,51,145
0,518,8345,3020,4,51,235,8139,3285,14,145,145,8347,1056,12,64,51
0,517,9111,3152,4,235,145,8010,3020,14,51,235,9105,3285,12,64,245
0,516,8451,3373,4,157,145,8446,3047,12,51,51,8437,3379,14,235,245
0,498,8014,3006,4,497,145,9111,3508,7,245,51,8008,3031,12,64,235
0,497,8473,3117,4,498,51,8465,3050,14,245,412,8453,2055,3,157,157
0,432,8112,3117,4,51,64,8126,3092,14,145,157,8105,3107,3,157,145
0,429,9104,3006,4,412,498,9111,3153,7,64,145,8135,3085,1,245,157
0,427,8347,3117,4,157,145,8321,3504,11,412,13,8214,3174,3,81,81
0,421,9111,1400,11,235,64,9923,3071,4,145,51,8105,3047,14,51,235


In [6]:
stats_df
champion_data2 = champion_data
stats_df = stats_df.rename(columns={"id": "key"})

In [ ]:
new_champion = pd.merge(champion_data, stats_df, on='key', how='outer')
new_champion.to_csv('new_champion_V1(corrected).csv')